# Project Week 2

Emiel Steegh   - s1846388  
Freek Nijweide - s1857746

# Project Description

Description etc etc

In [ ]:
# The following includes are needed to work with graphs and display solutions.
from __future__ import division
import networkx as nx
from IPython.core.display import display
from _dbus_bindings import Dictionary
%matplotlib inline

import numpy as np

import matplotlib.pyplot as plt
import utils
import graphs

import pandas as pd

plt.rcParams["figure.figsize"] = [12,8]

print("imports done")

## Assignment 1

text

## Assignment 2

text

## Assignment 3

In [ ]:
# code

## Assignment 4

There are three different ranking systems discussed in this week's chapters and lectures.
The first is an inDegree-based ranking system. The ranking of a page depends on how many pages link to it.
This system is easily fooled, and dead-end pages receive incredibly high rankings.

The second ranking system discussed is Google's PageRank. This system expands upon the idea of inDegree rankings,
by having a node "spread" its incoming ranking over its outgoing links. This can also include a degree of randomness,
where a node will spread some of its incoming ranking over *all* nods (to avoid spider traps and dead ends).

The third ranking system discussed HITS, based on the hub-authority model. In this model, a page is considered a good hub
if it links to good authorities (pages with good content that people want to see). A page is considered a good
authority if it is linked to by proper hubs. The page's authority ranking is the one actually used in the search.


# write equations
# implementeer ze
# leg code uit
# leg resultaten uit

In [ ]:
# Generate a graph we will use in the next exercise
graph = nx.DiGraph()

graph.add_edge('Google', 'Facebook')
graph.add_edge('Facebook', 'UTwente')
graph.add_edge('UTwente', 'Google')
graph.add_edge('Google', 'Reddit')
graph.add_edge('Reddit', 'Trap')
graph.add_edge('Spider','Trap')
graph.add_edge('Trap','Spider')
graph.add_edge('Reddit', 'Apple')
graph.add_edge('Reddit', 'Wikipedia')
graph.add_edge('Apple', 'Uber')
graph.add_edge('Uber', 'Twitter')
graph.add_edge('Twitter', 'Bing')
graph.add_edge('Wikipedia', 'Twitter')
graph.add_edge('Bing', 'Wikipedia')
graph.add_edge('Apple', 'Wikipedia')
graph.add_edge('Bing','Facebook')
graph.add_edge('UTwente','Dead end')
graph.add_edge('IN node','Apple')
graph.add_edge('Spam 1', 'Indegree Spam main page')
graph.add_edge('Spam 2', 'Indegree Spam main page')
graph.add_edge('Spam 3', 'Indegree Spam main page')
graph.add_edge('Spam 4', 'Indegree Spam main page')
graph.add_edge('Spam 5', 'Indegree Spam main page')
graph.add_edge('Spam 6', 'Indegree Spam main page')
display(utils.draw(graph))


# g.view()

In [131]:
def generate_M_and_v(graph):
    #type: (nx.DiGraph) -> any
    M = nx.to_numpy_matrix(graph) #type: np.matrix
    M.transpose()
    M_T = M.transpose()
    v = list(graph.nodes)
    return M,v

def generate_indegree_score(graph):    
    in_degrees = []
    for node in graph.nodes:
        if type(graph) == nx.classes.digraph.DiGraph:
            in_degrees.append(graph.in_degree(node))
        else:
            in_degrees.append(graph.degree(node))
    return in_degrees
    
def order_nodes_by_rank(nodes, rank):    
    order = np.argsort(rank)[::-1] #sort in degrees in descending order, return indices
    sorted_nodes = [nodes[i] for i in order] #Human readable sorted list of nodes
    ordinal_rank = [list(order).index(i) for i in range(len(order))] # Ordinal rank as specified in exercise
    return sorted_nodes,ordinal_rank

def ordinal_difference(a, b):
    if type(a) == list:
        a = np.array(a,dtype=float)
    if type(b) == list:
        b = np.array(b,dtype=float)
    return sum(abs(a-b))

def hits(M):
    if type(M) != np.matrix:
        M = nx.adjacency_matrix(M)
    v=np.ones(len(M),dtype=float)
    change_was_made = True
    h=np.copy(v)
    a=np.copy(v)
    while change_was_made:
        #calculate new v for hub and authority
        # M remains unchanged
        previous_a = np.copy(a)
        previous_h = np.copy(h)
        
        a = np.array(np.dot(M.transpose(),h)).flatten()
        a /= a.max()

        h = np.array(np.dot(M,a)).flatten()
        h /= h.max()
        change_was_made = ((abs(a-previous_a).max() ) > 0) or ((abs(h-previous_h).max() ) > 0) 
    
    return a,h

def pagerank(graph,beta):
    #type: (nx.DiGraph, float) -> any
    M,nodes = generate_M_and_v(graph)
    for i in range(len(nodes)):
        node=nodes[i]
        if graph.out_degree(node) > 0:
            M[i] /= graph.out_degree(node)
    M=M.T # We were working with adjacency matrix. The transition matrix is a transpose of this, where each
    #       number is divided by the out-degree of the node the edge is coming from
    original_v=np.ones(len(M),dtype=float)/len(M)
    v = np.copy(original_v)
    change_was_made = True
    while change_was_made:
        previous_v = np.copy(v)
        first_term = beta* (np.array(np.dot(M,v)).flatten())
        second_term = np.dot((1-beta),original_v)
        v = first_term + second_term
        change_was_made = ((abs(v-previous_v).max() ) > 0.00000000000001) #This 0.00... number was experimentally chosen. Smaller values seemed not to converge, for some graphs 
    
    return v

M,v = generate_M_and_v(graph)
print "nodes and adjacency matrix"
print v
print(M)

print "\n"

in_degrees = generate_indegree_score(graph)
human_readable, order = order_nodes_by_rank(v, in_degrees)
print "Solution to in_degree ranking:"
print "Human readable ranking (nodes sorted by their rank): " + str(human_readable)
print "Ordinal ranking: " + str(order)
print "\n"

a , h = hits(M)
human_readable, order = order_nodes_by_rank(v, a)
print "Our solution to HITS ranking"
print "Human readable ranking: " + str(human_readable)
print "Ordinal ranking: " + str(order)

print "\n"

print "HITS ranking (networkx implementation, for reference)"
(real_h_dict, real_a_dict) = nx.hits(graph,max_iter=100000)

difference = []
real_a=[]
for i in range(len(v)):
    node = v[i]
    real_a.append(real_a_dict[node])
    difference.append(abs(real_a_dict[node]-a[i]))
    
real_human_readable, real_order = order_nodes_by_rank(v, real_a)
print "Human readable ranking: " + str(real_human_readable)
print "Ordinal ranking: " + str(real_order)
print "Ordinal ranking difference between our implementation and networkx: " + str (ordinal_difference(order,real_order))

print "\n"

page_rank_list = pagerank(graph,0.85)
human_readable, order = order_nodes_by_rank(v, page_rank_list)
print "Our solution to pagerank"
print "Human readable ranking: " + str(human_readable)
print "Ordinal ranking: " + str(order)

print "\n"

# TODO do this for the different graphs in c, and print difference as mentioned in B

nodes and adjacency matrix
['Uber', 'Reddit', 'Google', 'Apple', 'Spam 5', 'Spam 1', 'Twitter', 'Spam 6', 'Wikipedia', 'UTwente', 'Spam 3', 'Spider', 'IN node', 'Facebook', 'Trap', 'Spam 4', 'Indegree Spam main page', 'Spam main page', 'Spam 2', 'Bing', 'Dead end']
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 

KeyboardInterrupt: 